In [23]:
# Packages
############################
'''
Template for the 4th assignment
Student: Fabian Gobet
'''
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

import torch
import torch.nn as nn
import math
from accelerate import Accelerator
import random
import re
from nltk.tokenize.treebank import TreebankWordTokenizer
import statistics
import pickle
import os
import matplotlib.pyplot as plt
import collections
import itertools
import numpy as np
from torch.utils.data import DataLoader, random_split

In [24]:
# Classes
############################
# Vocabulary class
class Vocabulary:
    '''
    Class for dealing with our corpus
    '''

    def __init__(self, name, sentences):
        """
        Args:
            name (str): name of the language
            pairs (list): list of pairs of sentences
        """
        self.name = name
        self.word2index = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2}
        self.index2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>"}
        for s in sentences:
            self.add_sentence(s)


    def add_word(self, word):
        '''
        Add a word to the vocabulary
        :param word: a string
        '''
        # TODO: add the word to the vocabulary
        if not word in self.word2index:
            self.word2index[word] = len(self.word2index)
            self.index2word[len(self.index2word)] = word


    def add_sentence(self, sentence):
        '''
        Add a sentence to the vocabulary
        :param sentence: list of strings (words)
        '''
        # TODO add the sentence to the vocabulary, this method will call the add_word method
        for word in sentence:
            self.add_word(word)


# Dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, vocabulary, pairs_refs, sentences):
        # TODO We want vocabulary and pairs to be attributes of the class
        self.vocabulary = vocabulary
        self.pairs = pairs_refs
        self.sentences = sentences

    def __len__(self):
        # TODO how many pairs do we have?
        return len(self.pairs)

    def __getitem__(self, ix):
        # TODO returns two tensors (question, answer) of the pair at index ix
        # TODO the tensors should be of type torch.tensor and should contain integers (word indices)
        q,a = self.pairs[ix]
        q = torch.tensor([self.vocabulary.word2index[word] for word in self.sentences[q]])
        a = torch.tensor([self.vocabulary.word2index[word] for word in ["<SOS>"]+self.sentences[a]])
        return q,a


class PositionalEncoding(nn.Module):
    '''
    Adapted from
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    '''
    def __init__(self, d_model, dropout=0.0, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.max_len = max_len

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()
                             * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        try:
            assert x.size(0) < self.max_len
        except:
            print("The length of the sequence is bigger than the max_len of the positional encoding. Increase the max_len or provide a shorter sequence.")
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=512, pad_id=0, encoder_layers=6, decoder_layers=6, dim_feedforward=2048, num_heads=8, dropout_transformer=0.1, dropout_posenconding=0):
        super().__init__()

        # TODO add an embedding layer
        self.embedding = nn.Embedding(vocab_size, d_model)

        # TODO add a positional encoding layer
        self.pos_encoder = PositionalEncoding(d_model,dropout=dropout_posenconding)

        # TODO add a transformer layer, you can use nn.Transformer. You can use the default values for the parameters, but what about batch_first?
        self.transformer = nn.Transformer(d_model=d_model, nhead=num_heads, num_encoder_layers=encoder_layers, num_decoder_layers=decoder_layers, dim_feedforward=dim_feedforward, dropout=dropout_transformer, batch_first=True)

        # TODO add a linear layer. Note: output should be probability distribution over the vocabulary
        self.linear = nn.Linear(d_model, vocab_size)

        # Stuff you may need
        self.vocab_size = vocab_size
        self.pad_id = pad_id
        self.num_heads = num_heads

    def create_padding_mask(self, x, pad_id=0):
        # TODO create a boolean mask for the <PAD> tokens
        return x.eq(pad_id)

    def forward(self, src, tgt):
        # S is the source sequence length, T is the target sequence length, N is the batch size, E is the feature number
        # src: (N, S)
        # tgt: (N, T)
        # src_pad_mask: (N, S)
        # tgt_pad_mask: (N, T)
        # mask the future : (N * num_heads, T, T)

        src_pad_mask = self.create_padding_mask(src, self.pad_id) # (N, S)
        tgt_pad_mask = self.create_padding_mask(tgt, self.pad_id) # (N, T)

        src = self.embedding(src)
        tgt = self.embedding(tgt)

        src = self.pos_encoder(src)  # (N, S, E)
        tgt = self.pos_encoder(tgt) # (N, T, E)

        # Mask the memory
        memory_key_padding_mask = src_pad_mask  # (N, S)

        # Mask the future
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.size(1), dtype=torch.bool).to(tgt.device) # (T, T)

        # Expand to make it N * num_heads, T, T
        tgt_mask = tgt_mask.unsqueeze(0).repeat(tgt.size(0) * self.num_heads, 1, 1) # (N, T, T)

        # Transformer
        output = self.transformer(src, tgt, tgt_mask=tgt_mask, src_key_padding_mask=src_pad_mask,tgt_key_padding_mask=tgt_pad_mask, memory_key_padding_mask=memory_key_padding_mask) # (N, T, E)
        # Linear layer
        output = self.linear(output) # (N, T, V)
        return output

class EarlyStopper:
    def __init__(self, patience):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        self.best_model = None

    def check_early_stop(self, current_loss, model):
        if current_loss < self.best_loss:
            self.best_loss = current_loss
            self.counter = 0
            self.best_model = model.state_dict()
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        return self.early_stop

In [29]:
# Methods new
############################
def clear_punctuation(s):
    '''
    This function removes all the punctuation from a sentence and insert a blank between any letter and !?.
    :param s: a string
    :return: the "cleaned" string
    '''
    # Remove all the character that are not letters, puntuation or numbers
    s = re.sub(r"[^a-zA-Z.!?,']+", r" ", s)
    # Insert a blank between any letter and !?. using regex
    #s = re.sub(r"([a-zA-Z])([!?.])", r"\1 \2", s)
    return s


def pickle_dump(obj, PATH, name):
    '''
    Save an object to a pickle file
    :param obj: object to save
    :param path: path to the pickle file
    '''
    if not os.path.exists(PATH):
        os.makedirs(PATH)

    with open(PATH+name, 'wb') as f:
        pickle.dump(obj, f)

def pickle_load(PATH):
    '''
    Load an object from a pickle file
    :param path: path to the pickle file
    :return: the loaded object
    '''
    with open(PATH, 'rb') as f:
        obj = pickle.load(f)
    return obj

def print_random_elements(collection, k=5):
    '''
    Print k random elements from a collection
    :param collection: list of elements
    :param k: number of elements to print
    '''
    random_elements = random.sample(collection, k=k)
    for e in random_elements:
        print(e)

def collate_fn(batch,pad_idx):
  data, targets = zip(*batch)
  padded_data = nn.utils.rnn.pad_sequence(data, batch_first=True, padding_value=pad_idx)
  padded_targets = nn.utils.rnn.pad_sequence(targets, batch_first=True, padding_value=pad_idx)
  return padded_data, padded_targets

def checkpoint(model,optimizer,path='./MyFiles/',save_name=None):
  dic = {
      'model_state' : model.state_dict(),
      'optimizer' : optimizer.state_dict()
  }
  if save_name is not None:
    torch.save(dic, path+save_name+".pt")
  return dic

def load_checkpoint(path):
    checkpoint = torch.load(path)
    model_state = checkpoint['model_state']
    optimizer_state = checkpoint['optimizer']
    return model_state,optimizer_state

def check_content_txt_files(convo_path, lines_path, num_elements):
    '''
    Check the content of the files
    :param convo_path: path to the movie_conversations.txt file
    :param lines_path: path to the movie_lines.txt file
    :param num_elements: number of elements to print
    '''
    # Inspect movie_conversations.txt
    with open(convo_path, 'r') as conv_file:
        for i in range(num_elements):
            line = conv_file.readline()

    # Inspect movie_lines.txt
    with open(lines_path, 'r') as lines_file:
        for i in range(num_elements):
            line = lines_file.readline()

def get_reference_pairs(convo_path):
    '''
    Get the reference pairs
    :return: a list of pairs of references of sentences
    '''
    ref_pairs = []
    with open(convo_path, 'r') as conv_file:
        for line in conv_file:
            conversation = line.strip().split(' +++$+++ ')[-1][1:-1].replace("'", "").split(",")
            for i in range(len(conversation) - 1):
                ref_pairs.append((conversation[i].strip(), conversation[i+1].strip()))
    return ref_pairs
    

def normalize_sentences(path_lines):
    lines_dict = {}
    empty_lines = 0
    with open(path_lines, 'r', encoding='cp1252') as lines_file:
        for full_line in lines_file:
            line_split = full_line.split(' +++$+++ ')
            line = line_split[-1]
            line, is_line_empty = process_sentence(line)
            if is_line_empty == 1:
                empty_lines += 1
            else:
                lines_dict.update({line_split[0] : line})
    return lines_dict, empty_lines

def process_sentence(line):
    is_line = 0
    line = line.replace('\n','').replace("<u>","").replace("</u>","").lower()
    line = re.sub('-+','',line)
    line = re.sub(r"[^a-zA-Z.!?,']+", r" ", line)
    line = re.sub(r"i'm", "i am", line, flags=re.I)
    line = re.sub(r"it's", "it is", line, flags=re.I)
    line = re.sub(r"he's", "he is", line, flags=re.I)
    line = re.sub(r"she's", "she is", line, flags=re.I)
    line = re.sub(r"can't", "can not", line, flags=re.I)
    line = re.sub(r"that's", "that is", line, flags=re.I)
    line = re.sub(r"there's", "there is", line, flags=re.I)
    line = re.sub(r"what's", "what is", line, flags=re.I)
    line = re.sub(r"where's", "where is", line, flags=re.I)
    line = re.sub(r"how's", "how is", line, flags=re.I)
    line = re.sub(r'\.+(?:\s*\.{1,})+', '...', line, flags=re.I)
    line = re.sub(r'!+(?:\s*!{1,})+', '!', line, flags=re.I)
    line = re.sub(r"\'ll", " will", line, flags=re.I)
    line = re.sub(r"\'em", " them", line, flags=re.I)
    line = re.sub(r"\'s", " is", line, flags=re.I)
    line = re.sub(r"\'ve", " have", line, flags=re.I)
    line = re.sub(r"\'re", " are", line, flags=re.I)
    line = re.sub(r"\'d", " would", line, flags=re.I)
    line = re.sub(r"n't", " not", line, flags=re.I)
    line = re.sub(r"won't", "will not", line, flags=re.I)
    line = re.sub(r"can't", "cannot", line, flags=re.I)
    line = line.strip()
    if line == '' or line.isspace():
        is_line += 1
    else:
        line = TreebankWordTokenizer().tokenize(line)
        if line[-1] not in ['.','!','?','...']:
            line.append('.')
    return line, is_line

def get_valid_refs(ref_pairs, lines):
    valid_refs = set()
    for p in ref_pairs:
        if p[0] in lines and p[1] in lines:
            valid_refs.add(p[0])
            valid_refs.add(p[1])
    return valid_refs

def generate_primitive_valid_pairs(all_ref_pairs, lines):
    chosen_sentences = {}
    chosen_ref_pairs = []

    for p in all_ref_pairs:
        if p[0] in lines and p[1] in lines:
            chosen_sentences.update({p[0]: lines[p[0]]+['<EOS>'], p[1]: lines[p[1]]+['<EOS>']})
            chosen_ref_pairs.append(p)
    return chosen_sentences, chosen_ref_pairs

def eliminate_long_sentences(chosen_sentences, chosen_ref_pairs, max_length):
    rule_out_sentences_refs = set()
    chosen_sentences2 = {}
    chosen_ref_pairs2 = []

    for k,v in chosen_sentences.items():
        if len(v) > max_length:
            rule_out_sentences_refs.add(k)

    for p in chosen_ref_pairs:
        if p[0] not in rule_out_sentences_refs and p[1] not in rule_out_sentences_refs:
            chosen_sentences2.update({p[0]: chosen_sentences[p[0]]})
            chosen_sentences2.update({p[1]: chosen_sentences[p[1]]})
            chosen_ref_pairs2.append(p)

    return chosen_sentences2,chosen_ref_pairs2,rule_out_sentences_refs

def eliminate_sentences_with_rare_words(chosen_sentences, chosen_ref_pairs, rule_out_words):
    rule_out_sentences_refs = set()
    chosen_sentences2 = {}
    chosen_ref_pairs2 = []

    for k,v in chosen_sentences.items():
        if any(word in rule_out_words for word in v):
            rule_out_sentences_refs.add(k)

    for p in chosen_ref_pairs:
        if p[0] not in rule_out_sentences_refs and p[1] not in rule_out_sentences_refs:
            chosen_sentences2.update({p[0]: chosen_sentences[p[0]]})
            chosen_sentences2.update({p[1]: chosen_sentences[p[1]]})
            chosen_ref_pairs2.append(p)

    return chosen_sentences2,chosen_ref_pairs2,rule_out_sentences_refs

def count_words(chosen_ref_pairs, chosen_sentences):
    word_counts = {}
    for p in chosen_ref_pairs:
        for r in p:
            for w in chosen_sentences[r]:
                if w in word_counts:
                    word_counts[w] += 1
                else:
                    word_counts[w] = 1
    num_words = sum(word_counts.values())
    return word_counts, num_words


def extract_sentences_from_refs(chosen_ref_pairs, chosen_sentences):
    chosen_sentences2 = {}
    for p in chosen_ref_pairs:
        chosen_sentences2.update({p[0]: chosen_sentences[p[0]]})
        chosen_sentences2.update({p[1]: chosen_sentences[p[1]]})
    return chosen_sentences2 


def create_pairs(path='./MyFiles/',savename="result",max_length=26,word_frequency_discard=10,verbose=True):

    if verbose:
        check_content_txt_files('./Data/movie_conversations.txt', './Data/movie_lines.txt', 5)

    all_ref_pairs = get_reference_pairs('./Data/movie_conversations.txt')

    if verbose:
        print_random_elements(all_ref_pairs)

    lines,empty_lines = normalize_sentences('./Data/movie_lines.txt')

    if verbose:
        print('Number of empty lines: {}'.format(empty_lines))
        print_random_elements(list(lines.values()))

    chosen_sentences, chosen_ref_pairs = generate_primitive_valid_pairs(all_ref_pairs, lines)

    if verbose:
        print_random_elements(chosen_ref_pairs)

        # Filter out the sentences that are too long
        # Compute the length of each sentence
        # Compute the mean and standard deviation for sentence lengths
        sentence_lengths = []
        for p in chosen_ref_pairs:
            sentence_lengths.append(len(chosen_sentences[p[0]]))
            sentence_lengths.append(len(chosen_sentences[p[1]]))

        mean_length = statistics.mean(sentence_lengths)
        std_dev = statistics.stdev(sentence_lengths)

        print('Mean sentence length: {}'.format(mean_length))
        print('Standard deviation: {}'.format(std_dev))
        print('Max sentence length: {}'.format(max(sentence_lengths)))
        print('Min sentence length: {}'.format(min(sentence_lengths)))

        # Plot the histogram with logarithmic scale on the y-axis
        plt.hist(sentence_lengths, density=True, bins=40)
        plt.xlabel('Sentence Length')
        plt.ylabel('Frequency (log scale)')
        plt.title('Sentence Length Distribution')
        plt.yscale('log')  # Set y-axis scale to logarithmic
        plt.axvline(x=mean_length, color='r', linestyle='--', label='Mean')  # Add vertical line for the mean
        plt.legend()
        plt.show()

        # Compute the frequency of each sentence length
        length_counts = {}
        for length in sentence_lengths:
            if length in length_counts:
                length_counts[length] += 1
            else:
                length_counts[length] = 1

        # Sort the sentence lengths in ascending order
        sorted_lengths = sorted(length_counts.keys())

        # Compute the accumulated frequency percentage
        total_sentences = len(sentence_lengths)
        accumulated_percentage = 0
        percentage_values = []
        for length in sorted_lengths:
            frequency = length_counts[length]
            percentage = (frequency / total_sentences) * 100
            accumulated_percentage += percentage
            percentage_values.append(accumulated_percentage)

        # Plot the sentence lengths and their accumulated frequency percentage
        plt.plot(sorted_lengths, percentage_values)
        plt.xlabel('Sentence Length')
        plt.ylabel('Accumulated Frequency Percentage')
        plt.title('Sentence Length Distribution')
        plt.show()

    initial_num_sentences = len(chosen_sentences)
    initial_num_pairs = len(chosen_ref_pairs)
    chosen_sentences2,chosen_ref_pairs2,rule_out_sentences_refs = eliminate_long_sentences(chosen_sentences, chosen_ref_pairs, max_length) 

    if verbose:
        print('Initial number of sentences: {}'.format(initial_num_sentences))
        print('Current number of senteces: {}'.format(initial_num_sentences-len(rule_out_sentences_refs)))
        print('Initial number of pairs: {}'.format(initial_num_pairs))
        print('Current number of pairs: {}'.format(initial_num_pairs-len(chosen_ref_pairs2)))


    word_counts, num_words = count_words(chosen_ref_pairs2, chosen_sentences2)

    if verbose:
        print('Number of words: {}'.format(num_words))
        print('Number of unique words: {}'.format(len(word_counts)))

        plt.plot(range(len(word_counts)), list(word_counts.values()))
        plt.xlabel('Word index')
        plt.ylabel('Word frequency')
        plt.suptitle('Word frequency distribution')
        plt.title('Frequency per word')
        plt.show()

        # Compute the mean and standard deviation for word counts
        mean_value = statistics.mean(word_counts.values())
        print("Mean value:", mean_value)
        print("Max value:", max(word_counts.values()))
        print("Min value:", min(word_counts.values()))

        # Plot the distribution of word counts < mean
        dict_lower_mean = {k: v for k, v in word_counts.items() if v < mean_value}
        sorted_dict_lower_mean = dict(sorted(dict_lower_mean.items(), key=lambda x: x[1], reverse=True))

        plt.plot(range(len(sorted_dict_lower_mean)), sorted_dict_lower_mean.values())
        plt.xlabel('Word index')
        plt.ylabel('Word count')
        plt.suptitle('Word count distribution')
        plt.title('Words with count < mean')
        plt.show()

    # Frequency threshold and filter out the words that are too rare
    rule_out_words = [k for k, v in word_counts.items() if v < word_frequency_discard]
    chosen_sentences3,chosen_ref_pairs3,rule_out_sentences_refs = eliminate_sentences_with_rare_words(chosen_sentences2, chosen_ref_pairs2, rule_out_words)

    if verbose:
        print('Current number of senteces: {} ({:.2f}% of total)'.format(initial_num_sentences-len(chosen_sentences3),(initial_num_sentences-len(chosen_sentences3))/initial_num_sentences*100))
        print('Current number of pairs: {} ({:.2f}% of total)'.format(initial_num_pairs-len(chosen_ref_pairs3),(initial_num_pairs-len(chosen_ref_pairs3))/initial_num_pairs*100))

    # Save the pairs to a pickle file


    pickle_dump(chosen_sentences3,path,savename+"_sentences.pkl")
    pickle_dump(chosen_ref_pairs3,path,savename+"_ref_pairs.pkl")

    return chosen_sentences3,chosen_ref_pairs3


def train_with_gradient_accumulation(epochs, model, criterion, optimizer, train_loader, device, val_loader, accumulation_batches, lr_scheduler, print_every_n_batches=32):

    assert  print_every_n_batches % accumulation_batches == 0, "print_every_n_batches must be a multiple of accumulation_batches"
    assert  len(train_loader) % print_every_n_batches == 0, "print_every_n_batches must be a multiple of len(train_loader)"

    train_losses = []
    val_losses = []
    vocab = train_loader.dataset.vocabulary
    optimizer.zero_grad()
    running_loss = 0

    for epoch in range(epochs):
        for i, (data, targets) in enumerate(train_loader):
          model.train()
          data = data.to(device)
          targets = targets.to(device)
          outputs = model(data,targets[:,:-1])
          loss = criterion(outputs.reshape(-1,outputs.size(-1)), targets[:,1:].reshape(-1))
          loss = loss / accumulation_batches
          running_loss += loss.item()
          loss.backward()

          if (i+1) % accumulation_batches == 0 or i + 1 == len(train_loader):
              train_losses.append(running_loss)
              running_loss = 0
              optimizer.step()
              if lr_scheduler:
                lr_scheduler.step()
              optimizer.zero_grad()

          if (i+1) % print_every_n_batches == 0 or i + 1 == len(train_loader):
              model.eval()
              total_loss = 0  
              with torch.no_grad():
                  for data, targets in val_loader:
                      data = data.to(device)
                      targets = targets.to(device)
                      outputs = model(data,targets[:,:-1])
                      loss = criterion(outputs.reshape(-1,outputs.size(-1)), targets[:,1:].reshape(-1))
                      total_loss += loss.item()
                  val_losses.append(total_loss / len(val_loader))  
              model.train()

              random_index = random.randint(0, len(targets)-1)
              target_sentence = " ".join([vocab.index2word[idx.item()] for idx in targets[random_index][1:]])
              output_sentence = " ".join([vocab.index2word[idx.item()] for idx in outputs.argmax(dim=-1)[random_index]])

              print(f"Epoch: {epoch+1}/{epochs}, Batch: {i+1}/{len(train_loader)}")
              print(f"Train Loss: {train_losses[-1]:.4f}")
              print(f"Validation Loss: {val_losses[-1]:.4f}")
              print(f"Random Target Sentence: {target_sentence}")
              print(f"Random Output Sentence: {output_sentence}\n")

    return train_losses, val_losses, model

In [34]:
# !!! Don't change the seed !!!
torch.manual_seed(42)
# !!!!!!


# Load pairs
PATH = './MyFiles/'
chosen_sentences = pickle_load(PATH+'result_sentences.pkl')
chosen_ref_pairs = pickle_load(PATH+'result_ref_pairs.pkl')

# Pipeline parameters
############################
# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128 #128 batches
accumulation_batches = 16 #64 updates per epoch
rand_sample_num = 40960
learning_rate = 1e-4
d_model = 512 #768
encoder_layers = 6
decoder_layers = 5
feed_forward_dim = 2048
nheads = 8
dropout_transformer = 0.2
dropout_posenconding = 0
patience = 3
epochs = 10
print_every_n_batches = 32

# Randomly sample pairs and splits
rand_sample_num = min(rand_sample_num,len(chosen_ref_pairs))
chosen_ref_pairs = random.sample(chosen_ref_pairs,rand_sample_num)
train_size = (int(0.8 * rand_sample_num)//batch_size)*batch_size
val_size = (rand_sample_num-train_size)//2
test_size = rand_sample_num-train_size-val_size
train_pairs, test_pairs, val_pairs = random_split(chosen_ref_pairs,[train_size,val_size,test_size])

# Vocabulary, Dataset and Dataloader
vocab = Vocabulary("English",chosen_sentences.values())
train_dataset = Dataset(vocab,train_pairs,extract_sentences_from_refs(train_pairs, chosen_sentences))
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,collate_fn=lambda batch: collate_fn(batch,vocab.word2index["<PAD>"]))
val_dataset = Dataset(vocab,val_pairs,extract_sentences_from_refs(val_pairs, chosen_sentences))
val_loader = DataLoader(val_dataset,batch_size=batch_size,shuffle=False,collate_fn=lambda batch: collate_fn(batch,vocab.word2index["<PAD>"]))


# Pipeline initialization
############################
# Model, criterion, optimizer and scheduler
model = TransformerModel(vocab_size=len(vocab.word2index),d_model=d_model,pad_id=vocab.word2index["<PAD>"],
                        encoder_layers=encoder_layers,decoder_layers=decoder_layers,dim_feedforward=feed_forward_dim,
                        num_heads=nheads,dropout_transformer=dropout_transformer, dropout_posenconding=dropout_posenconding).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9, verbose=False)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=10, factor=0.8)
stopper = EarlyStopper(patience=patience)

In [27]:
print(len(train_loader)/accumulation_batches) # updates/logs per epoch
print(len(train_loader)/print_every_n_batches) # logs per epoch

64.0
8.0


In [35]:

train_losses, val_losses, model = train_with_gradient_accumulation(epochs, model, criterion, optimizer, train_loader,
                                                                    device, val_loader, accumulation_batches, lr_scheduler)

Epoch: 1/10, Batch: 16/256
Train Loss: 9.5964
Validation Loss: 9.6061
Random Target Sentence: nothing . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Random Output Sentence: common conway conway conway punish punish punish punish punish punish punish punish punish punish punish punish punish punish punish punish punish punish punish punish punish

Epoch: 1/10, Batch: 32/256
Train Loss: 6.0273
Validation Loss: 5.3162
Random Target Sentence: your valet , sir . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Random Output Sentence: <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Epoch: 1/10, Batch: 48/256
Train Loss: 4.6723
Validation Loss: 4.1133
Random Target Sentence: rules are made to be broken . <EOS> <PAD> <PAD> <PAD> <P

Epoch: 10/10, Batch: 256/256
Train Loss: 1.5662
Validation Loss: 1.5334
Random Target Sentence: how much ? <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Random Output Sentence: i do ? <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128 #128 batches
accumulation_batches = 4 #64 updates per epoch
rand_sample_num = 40960
learning_rate = 8e-4
d_model = 512 #768
encoder_layers = 6
decoder_layers = 5
feed_forward_dim = 2048
nheads = 8
dropout_transformer = 0.2
dropout_posenconding = 0
patience = 3
epochs = 10
print_every_n_batches = 32

In [ ]:
# Plot train losses and validation losses
num = epochs*np.ceil(len(train_loader)/print_every_n_batches)
plt.plot(np.linspace(1,num,len(train_losses)),train_losses, label='Train Loss')
plt.plot(np.linspace(1,num,len(val_losses)),val_losses, label='Validation Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Train Loss vs Validation Loss')
plt.legend()
plt.show()